<a href="https://colab.research.google.com/github/iGhostlp/Albus/blob/severus/Proyecto_BBVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [3]:
ls

Datasets/     spark-3.3.2-bin-hadoop3/     spark-3.3.2-bin-hadoop3.tgz.1
sample_data/  spark-3.3.2-bin-hadoop3.tgz  spark-3.3.2-bin-hadoop3.tgz.2


In [4]:
!tar xf spark-3.3.2-bin-hadoop3.tgz


In [5]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [6]:
!readlink -f $(which java) | sed "s:bin/java::"


/usr/lib/jvm/java-11-openjdk-amd64/


In [7]:
!echo $JAVA_HOME

In [8]:
!pip install -q findspark


In [9]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [11]:
import findspark


In [12]:
import pandas as pd


In [13]:
import matplotlib.pyplot as plt

In [14]:
findspark.init()


In [15]:
findspark.find()


'/content/spark-3.3.2-bin-hadoop3'

In [16]:
from pyspark.sql import SparkSession


In [17]:
from pyspark.sql.functions import col


In [18]:
spark = SparkSession.builder.master("local[*]").config('spark.sql.parquet.datetimeRebaseModeInRead','CORRECTED').getOrCreate()


In [19]:
spark

In [20]:
path_address = '/content/Datasets/address.snappy.parquet'
path_customer_basics = '/content/Datasets/customer_basics.snappy.parquet'
path_customer = '/content/Datasets/customer.snappy.parquet'
path_mails = '/content/Datasets/email.snappy.parquet'
path_phones = '/content/Datasets/phones.snappy.parquet'


In [21]:
df_address = spark.read.load(path_address, sep=',' , inferschema= 'true', header = 'true')

df_customerb = spark.read.load(path_customer_basics, sep = ',' , inferschema = 'true', header = 'true')

df_customer = spark.read.load(path_customer, sep = ',' , inferschema = 'true', header = 'true')

df_mail = spark.read.load(path_mails, sep = ',' , inferschema = 'true', header = 'true')

df_phones = spark.read.load(path_phones, sep = ',' , inferschema = 'true', header = 'true')



In [57]:
df_customer_address = df_address.join(df_customerb, ['customer_id'], how = 'left')


In [56]:
df_customer_address.show()

+-----------+-----------------------+-------------------+--------------+-----------+------------------+---------------------------+-----------------+-------------+---------------------+-----------------+-----------------------+----------+---------------+-----------+------------------+----------------------+-------------------------+------------------+---------------------+------------------------------+-----------------------+--------------------+---------------------+------------------+-------------------------+-------------------------+------------------+-----------------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+-----------------+---------------------+-------------+-------------+-

In [47]:
df_address.show()

+-----------+-----------------------+-------------------+--------------+--------------------+------------------+---------------------------+-----------------+-------------+---------------------+--------------------+-----------------------+----------+---------------+-----------+------------------+----------------------+-------------------------+------------------+---------------------+------------------------------+-----------------------+--------------------+---------------------+------------------+-------------------------+-------------------------+------------------+-----------------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+-----------------+---------------------+
|customer_id|add